In [1]:
#! git clone https://github.com/experiencor/raccoon_dataset.git
import pycocotools
import sys
import os

os.chdir("./tools")
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import pandas as pd
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import torchvision


from PIL import Image

import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import pandas as pd
import json
import math
import torch
#!export OMP_NUM_THREADS=1

## Insert data path
#Mandus 
#path_to_data_file = "../../Data/"
#path_to_csv = '../data/dataFrameMerged.csv'

# Eric 
path_to_data_file = "../../../RCCN-tutorial/trodo-v01/"
path_to_csv = '../data/dataFrameMerged.csv'

dataFrame = pd.read_csv(path_to_csv)    


In [12]:
dataFrame
!export OMP_NUM_THREADS=1

In [3]:

def filelist(root, file_type):
    """Returns a fully-qualified list of filenames under root directory"""
    return [os.path.join(directory_path, f) for directory_path, directory_name, 
            files in os.walk(root) for f in files if f.endswith(file_type)]

# Preparing the data 

In [4]:
def get_labels(img,idx):
    label = dataFrame["odometer_type"][idx]
    
    return label
    
def read_odometer_boxes(path_data_file, idx):
    boxes = []
    
    ymin = dataFrame["ymin"][idx]
    ymax = dataFrame["ymax"][idx]
    xmin = dataFrame["xmin"][idx]
    xmax = dataFrame["xmax"][idx]
    
    boxes.append([xmin, ymin, xmax, ymax])
            
    return boxes


In [5]:
class TrodoDataset(torch.utils.data.Dataset):
    def __init__(self, root, data_file, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = sorted(os.listdir(os.path.join(root, "images")))
        self.data_file = data_file
        #self.path_to_data_file = data_file
    def __getitem__(self, idx):
      # load images and bounding boxes
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        label = get_labels(self.imgs,idx)
        labels = torch.tensor((label,), dtype=torch.int64)
        
        box_list = read_odometer_boxes(self.data_file,idx)
        boxes = torch.as_tensor(box_list, dtype=torch.float32)
        
        image_id = torch.tensor([idx])
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:,0])
        
        iscrowd = torch.zeros((2,), dtype=torch.int64)
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target
    def __len__(self):
         return len(self.imgs)

In [6]:
dataset = TrodoDataset(root = path_to_data_file,
                         data_file= "../data/dataFrameMerged.csv")
#dataset.__getitem__(5)

#for e in dataset:
#    print(e)




# Download and adjust the model

In [13]:

def get_model(num_classes):
    # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)
    return model

In [8]:
def get_transform(train):
    transforms = []
   # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
      # during training, randomly flip the training images
      # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [9]:
# use our dataset and defined transformations
dataset = TrodoDataset(root= path_to_data_file,
                         data_file= "../data/dataFrameMerged.csv",
                         transforms = get_transform(train=True))
dataset_test = TrodoDataset(root= path_to_data_file,
                         data_file= "../data/dataFrameMerged.csv",
                              transforms = get_transform(train=False))
# split the dataset in train and test set
#torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-200])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-200:])
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True, num_workers=0,
                                          collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=5,
                                               shuffle=False, num_workers=0,
                                               collate_fn=utils.collate_fn)


print("We have: {} examples, {} are training and {} testing".format(len(indices), len(dataset), len(dataset_test)))

We have: 2389 examples, 2189 are training and 200 testing


# Training the model

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# our dataset has two classes only - digital or analog odometer
num_classes = 2
model = get_model(num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)


1024


In [15]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

torch.set_num_threads(1) 

num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch,
                   print_freq=1)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


Epoch: [0]  [  0/438]  eta: 12:44:14  lr: 0.000016  loss: 0.8310 (0.8310)  loss_classifier: 0.7375 (0.7375)  loss_box_reg: 0.0183 (0.0183)  loss_objectness: 0.0672 (0.0672)  loss_rpn_box_reg: 0.0080 (0.0080)  time: 104.6912  data: 0.1182


KeyboardInterrupt: 

In [46]:
### Save model ###
#time = date.time()
#os.mkdir("models/pytorch_odometer_classifier/")
#torch.save(model.state_dict(), "models/pytorch_odometer_classifier/model" + time)

(<PIL.Image.Image image mode=RGB size=650x417 at 0x7FF35AF81390>,
 {'boxes': tensor([[ 81.,  88., 522., 408.]]),
  'labels': tensor([1]),
  'image_id': tensor([0]),
  'area': tensor([141120.]),
  'iscrowd': tensor([0])})

## Making predictions with the model

In [ ]:
### Load Model ###

##loaded_model = get_model(num_classes = 2)
##loaded_model.load_state_dict(torch.load(models/pytorch_odometer_classifier/model" + time))